In [1]:
# Install geospatial libraries
!pip install osmnx geopandas folium requests gtfs-kit

# Import libraries
import osmnx as ox
import geopandas as gpd
import folium
import pandas as pd
import requests
import numpy as np

print("Libraries installed successfully!")

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.6/507.6 kB 25.2 MB/s eta 0:00:00
  Created wheel for json2html: filename=json2html-1.3.0-py3-none-any.whl size=7591 sha256=c92a9aba8d5d266a1ba52ad45a747a4fbb3128fb8c9ece13970e26bd9144fa70
  Stored in directory: /root/.cache/pip/wheels/91/96/d0/3cfff4cc9a9e307cfc2b8fe4846c56d714b78bbe5f6da211a5
Successfully built json2html
Libraries installed successfully!


In [2]:
# Get Foggy Bottom area data (UPDATED for newer OSMnx)
print("Extracting Foggy Bottom data...")

# Get neighborhood boundary
try:
    foggy_bottom_boundary = ox.geocode_to_gdf("Foggy Bottom, Washington, DC, USA")
    print(f" Foggy Bottom boundary found")
except:
    print(" Couldn't find exact boundary, using coordinates instead")
    foggy_bottom_center = (38.9006, -77.0481)

# FIXED: Use features_from_place instead of geometries_from_place coz latter is deprecated
try:
    hospitals = ox.features_from_place(
        "Foggy Bottom, Washington, DC, USA",
        tags={'amenity': 'hospital'}
    )
    print(f" Found {len(hospitals)} hospitals")
except Exception as e:
    print(f" Hospital search failed: {e}")
    hospitals = pd.DataFrame()

# Get metro/subway stations
try:
    metro_stations = ox.features_from_place(
        "Foggy Bottom, Washington, DC, USA",
        tags={'railway': 'station'}
    )
    print(f" Found {len(metro_stations)} metro stations")
except Exception as e:
    print(f" Metro station search failed: {e}")
    metro_stations = pd.DataFrame()

# Display what we found
if len(hospitals) > 0:
    print("\nHospitals found:")
    if 'name' in hospitals.columns:
        print(hospitals['name'].head())
    else:
        print("Hospital data found but no names available")

if len(metro_stations) > 0:
    print("\nMetro stations found:")
    if 'name' in metro_stations.columns:
        print(metro_stations['name'].head())
    else:
        print("Station data found but no names available")

# If no results, let's try a broader search
if len(hospitals) == 0 and len(metro_stations) == 0:
    print("\nTrying broader search for Washington DC...")
    try:
        dc_hospitals = ox.features_from_place(
            "Washington, DC, USA",
            tags={'amenity': 'hospital'}
        )
        print(f"Found {len(dc_hospitals)} hospitals in all of DC")
    except:
        print("Broader search also failed")

Extracting Foggy Bottom data...
 Foggy Bottom boundary found
 Hospital search failed: No matching features. Check query location, tags, and log.
 Metro station search failed: No matching features. Check query location, tags, and log.

Trying broader search for Washington DC...
Found 15 hospitals in all of DC


In [3]:
# Strategy 1: Try different hospital tags
print("=== Trying different hospital tags ===")

hospital_tags = [
    {'amenity': 'hospital'},
    {'healthcare': 'hospital'},
    {'healthcare': 'centre'},
    {'amenity': 'clinic'}
]

for i, tag in enumerate(hospital_tags):
    try:
        results = ox.features_from_place("Foggy Bottom, Washington, DC, USA", tags=tag)
        print(f"Tag {tag}: Found {len(results)} results")
        if len(results) > 0 and 'name' in results.columns:
            print(f"  Names: {list(results['name'].dropna().head())}")
    except:
        print(f"Tag {tag}: Failed")

print("\n=== Searching by coordinates around GWU Hospital ===")
# Strategy 2: Search directly around GWU Hospital coordinates
gwu_lat, gwu_lon = 38.9013, -77.0494  # Approximate GWU Hospital location

try:
    gwu_area = ox.features_from_point(
        (gwu_lat, gwu_lon),
        tags={'amenity': 'hospital'},
        dist=300  # 300m radius
    )
    print(f"Found {len(gwu_area)} hospitals near GWU coordinates")
    if len(gwu_area) > 0:
        print("Hospital names found:")
        if 'name' in gwu_area.columns:
            print(gwu_area['name'].dropna())
        else:
            print("No names in data, but geometry found")
except Exception as e:
    print(f"GWU area search failed: {e}")

print("\n=== Searching all of DC for 'GWU' or 'George Washington' ===")
# Strategy 3: Search all DC hospitals and filter for GWU
try:
    all_dc_hospitals = ox.features_from_place(
        "Washington, DC, USA",
        tags={'amenity': 'hospital'}
    )
    print(f"Total DC hospitals found: {len(all_dc_hospitals)}")

    if len(all_dc_hospitals) > 0 and 'name' in all_dc_hospitals.columns:
        gwu_hospitals = all_dc_hospitals[
            all_dc_hospitals['name'].str.contains('GWU|George Washington|University',
                                                case=False, na=False)
        ]
        print(f"GWU-related hospitals: {len(gwu_hospitals)}")
        if len(gwu_hospitals) > 0:
            print("GWU hospital names:")
            print(gwu_hospitals['name'].tolist())

        print("\nAll DC hospital names (first 10):")
        print(all_dc_hospitals['name'].dropna().head(10).tolist())

except Exception as e:
    print(f"DC-wide search failed: {e}")

=== Trying different hospital tags ===
Tag {'amenity': 'hospital'}: Failed
Tag {'healthcare': 'hospital'}: Failed
Tag {'healthcare': 'centre'}: Failed
Tag {'amenity': 'clinic'}: Failed

=== Searching by coordinates around GWU Hospital ===
Found 1 hospitals near GWU coordinates
Hospital names found:
element  id       
way      237887734    George Washington University Hospital
Name: name, dtype: object

=== Searching all of DC for 'GWU' or 'George Washington' ===
Total DC hospitals found: 15
GWU-related hospitals: 3
GWU hospital names:
['George Washington University Hospital', 'MedStar Georgetown University Hospital', 'Howard University Hospital']

All DC hospital names (first 10):
['Mule Hospital', 'Washington Hospital', 'DC General Hospital (historic)', 'Wasington DC VA Medical Center', 'George Washington University Hospital', "Children's National Hospital", 'United Medical Center', 'MedStar Georgetown University Hospital', 'Sibley Memorial Hospital', 'Hospital for Sick Children Pedia

In [4]:
# Get Metro stations for Foggy Bottom area
print("=== Finding Metro Stations ===")

# Try different tags for transit stations
station_tags = [
    {'railway': 'station'},
    {'public_transport': 'station'},
    {'railway': 'subway_entrance'},
    {'subway': 'yes'}
]

all_stations = []

for tag in station_tags:
    try:
        stations = ox.features_from_place("Foggy Bottom, Washington, DC, USA", tags=tag)
        print(f"Tag {tag}: Found {len(stations)} stations")
        if len(stations) > 0:
            all_stations.append(stations)
    except:
        print(f"Tag {tag}: Failed")

# Also search by coordinates around Foggy Bottom Metro
fb_lat, fb_lon = 38.9006, -77.0481

try:
    nearby_stations = ox.features_from_point(
        (fb_lat, fb_lon),
        tags={'railway': 'station'},
        dist=800  # 800m radius
    )
    print(f"\nFound {len(nearby_stations)} stations near Foggy Bottom coordinates")

    if len(nearby_stations) > 0 and 'name' in nearby_stations.columns:
        print("Station names:")
        station_names = nearby_stations['name'].dropna().unique()
        for name in station_names:
            print(f"  - {name}")

except Exception as e:
    print(f"Station coordinate search failed: {e}")

# Store the best stations data for mapping
if len(nearby_stations) > 0:
    foggy_stations = nearby_stations
    print(f"\n Using {len(foggy_stations)} stations for mapping")
else:
    print(" No stations found - will use manual coordinates")
    foggy_stations = pd.DataFrame()

=== Finding Metro Stations ===
Tag {'railway': 'station'}: Failed
Tag {'public_transport': 'station'}: Found 3 stations
Tag {'railway': 'subway_entrance'}: Failed
Tag {'subway': 'yes'}: Failed

Found 3 stations near Foggy Bottom coordinates
Station names:
  - Farragut West
  - Foggy Bottom–GWU
  - Farragut North

 Using 3 stations for mapping


In [5]:
# Create interactive map of Foggy Bottom
print("Creating Foggy Bottom map...")

# Center on Foggy Bottom
center_lat, center_lon = 38.9006, -77.0481
m = folium.Map(location=[center_lat, center_lon], zoom_start=15)

# Add hospitals (we know this works from previous cell)
gwu_lat, gwu_lon = 38.9013, -77.0494  # GWU Hospital coordinates
folium.Marker(
    [gwu_lat, gwu_lon],
    popup="🏥 George Washington University Hospital",
    tooltip="GWU Hospital - Major trauma center",
    icon=folium.Icon(color='red', icon='plus', prefix='fa')
).add_to(m)

# Add Foggy Bottom Metro Station
metro_lat, metro_lon = 38.9006, -77.0500  # Foggy Bottom-GWU Metro
folium.Marker(
    [metro_lat, metro_lon],
    popup=" Foggy Bottom-GWU Metro Station",
    tooltip="Blue/Orange/Silver Lines",
    icon=folium.Icon(color='blue', icon='train', prefix='fa')
).add_to(m)

# Add neighborhood center
folium.CircleMarker(
    [center_lat, center_lon],
    radius=10,
    popup="📍 Foggy Bottom Center",
    tooltip="Neighborhood center point",
    color='green',
    fill=True,
    fillColor='lightgreen'
).add_to(m)

# Add a circle showing walking distance (10 minutes = ~800m)
folium.Circle(
    [center_lat, center_lon],
    radius=800,
    popup="10-minute walk from center",
    color='lightblue',
    fill=True,
    fillOpacity=0.1
).add_to(m)

print("✅ Map created!")
print("🗺️  Displaying interactive map...")

# Display the map
m

Creating Foggy Bottom map...
✅ Map created!
🗺️  Displaying interactive map...


In [6]:
# Store Foggy Bottom data for later analysis
foggy_bottom_data = {
    'name': 'Foggy Bottom',
    'center': (38.9006, -77.0481),
    'hospitals': [
        {'name': 'George Washington University Hospital', 'coords': (38.9013, -77.0494)}
    ],
    'metro_stations': [
        {'name': 'Foggy Bottom-GWU', 'coords': (38.9006, -77.0500), 'lines': ['Blue', 'Orange', 'Silver']}
    ],
    'income_level': 'High',
    'notes': 'Hospital within walking distance, multiple metro lines'
}

print("✅ Foggy Bottom data saved!")
print(f"📊 Summary: {len(foggy_bottom_data['hospitals'])} hospitals, {len(foggy_bottom_data['metro_stations'])} metro stations")

✅ Foggy Bottom data saved!
📊 Summary: 1 hospitals, 1 metro stations


In [7]:
# ============================================================
# ============================================================
# NEW SECTION: VALIDATED HOSPITAL DATA
# Using Official DC.gov Hospital Registry
# (Previous OSM extraction had data quality issues)
# ============================================================
# ============================================================

In [9]:
# Load official DC hospital list (8 hospitals)
#read directly instead of creating csv
#using dc.gov
hospital_data = """hospital_id,hospital_name,hospital_type,address,ward
hospital_001,George Washington University Hospital,General Acute Care,900 23rd Street NW,2
hospital_002,Howard University Hospital,General Acute Care,2041 Georgia Avenue NW,1
hospital_003,MedStar Georgetown Hospital,General Acute Care,3800 Reservoir Road NW,2
hospital_004,MedStar Washington Hospital Center,General Acute Care,110 Irving Street NW,5
hospital_005,Providence Hospital,General Acute Care,1150 Varnum Street NE,5
hospital_006,Sibley Memorial Hospital,General Acute Care,5255 Loughboro Road NW,3
hospital_007,United Medical Center,General Acute Care,1310 Southern Avenue SE,8
hospital_008,Children's National Medical Center,Pediatric,111 Michigan Avenue NW,5"""

# Create DataFrame
#to read text as a file
from io import StringIO
hospitals_official = pd.read_csv(StringIO(hospital_data))

print(f"Loaded {len(hospitals_official)} validated hospitals")
hospitals_official

✓ Loaded 8 validated hospitals


,hospital_id,hospital_name,hospital_type,address,ward
0,hospital_001,George Washington University Hospital,General Acute Care,900 23rd Street NW,2
1,hospital_002,Howard University Hospital,General Acute Care,2041 Georgia Avenue NW,1
2,hospital_003,MedStar Georgetown Hospital,General Acute Care,3800 Reservoir Road NW,2
3,hospital_004,MedStar Washington Hospital Center,General Acute Care,110 Irving Street NW,5
4,hospital_005,Providence Hospital,General Acute Care,1150 Varnum Street NE,5
5,hospital_006,Sibley Memorial Hospital,General Acute Care,5255 Loughboro Road NW,3
6,hospital_007,United Medical Center,General Acute Care,1310 Southern Avenue SE,8
7,hospital_008,Children's National Medical Center,Pediatric,111 Michigan Avenue NW,5


In [11]:
# Geocode first hospital
first = hospitals_official.iloc[0]
address = f"{first['address']}, Washington, DC"

print(f"Geocoding: {first['hospital_name']}")
print(f"Address: {address}")

#Geocode address 1
lat, lon = ox.geocode(address)

print(f"✓ Lat: {lat:.6f}, Lon: {lon:.6f}")








Geocoding: George Washington University Hospital
Address: 900 23rd Street NW, Washington, DC
✓ Lat: 38.901377, Lon: -77.050616


In [20]:
#geocode all 8 hospitals
for index,hospital in hospitals_official.iterrows():
  address = f"{hospital['address']}, Washington, DC"
  print(f"Geocoding: {hospital['hospital_name']}")
  print(f"Address: {address}")
  lat, lon = ox.geocode(address)
  print(f"✓ Lat: {lat:.6f}, Lon: {lon:.6f}")
  hospitals_official.at[index,'latitude']=lat
  hospitals_official.at[index,'longitude']=lon


print(hospitals_official)



Geocoding: George Washington University Hospital
Address: 900 23rd Street NW, Washington, DC
✓ Lat: 38.901377, Lon: -77.050616
Geocoding: Howard University Hospital
Address: 2041 Georgia Avenue NW, Washington, DC
✓ Lat: 38.917834, Lon: -77.020100
Geocoding: MedStar Georgetown Hospital
Address: 3800 Reservoir Road NW, Washington, DC
✓ Lat: 38.911177, Lon: -77.073814
Geocoding: MedStar Washington Hospital Center
Address: 110 Irving Street NW, Washington, DC
✓ Lat: 38.929652, Lon: -77.015648
Geocoding: Providence Hospital
Address: 1150 Varnum Street NE, Washington, DC
✓ Lat: 38.944542, Lon: -76.993406
Geocoding: Sibley Memorial Hospital
Address: 5255 Loughboro Road NW, Washington, DC
✓ Lat: 38.936770, Lon: -77.108880
Geocoding: United Medical Center
Address: 1310 Southern Avenue SE, Washington, DC
✓ Lat: 38.835876, Lon: -76.984938
Geocoding: Children's National Medical Center
Address: 111 Michigan Avenue NW, Washington, DC
✓ Lat: 38.927021, Lon: -77.013979
    hospital_id                 

In [24]:
#export to csv file
hospitals_official.to_csv('dc_hospitals_geocoded.csv')
print(" CSV File Saved")

 CSV File Saved


In [25]:
# Check if file exists
import os
print(os.path.exists('dc_hospitals_geocoded.csv'))

True


In [29]:

# TODO: Calculate map center from  actual hospital coordinates
#dont use dc center as it is hard coding dc center so it will break if:
#move to baltimore/virginia


center_lat = hospitals_official['latitude'].mean()
center_lon = hospitals_official['longitude'].mean()

print(f"Map center: ({center_lat:.4f}, {center_lon:.4f})")

Map center: (38.9130, -77.0327)


In [42]:
hospital_map = folium.Map(location=[center_lat, center_lon], zoom_start=11)

In [47]:
#Add markers for all hospitals
for index,hospital in hospitals_official.iterrows():
    # Get coordinates and name
    lat = hospital['latitude']
    lon = hospital['longitude']
    name = hospital['hospital_name']

    # Add marker
    folium.Marker(
        location=[lat,lon],
        popup=name
    ).add_to(hospital_map)

# Display the updated map
hospital_map

In [45]:
# Count how many markers  added
print(f"Number of hospitals: {len(hospitals_official)}")
print("\nAll hospital names:")
for name in hospitals_official['hospital_name']:
    print(f"  - {name}")

Number of hospitals: 8

All hospital names:
  - George Washington University Hospital
  - Howard University Hospital
  - MedStar Georgetown Hospital
  - MedStar Washington Hospital Center
  - Providence Hospital
  - Sibley Memorial Hospital
  - United Medical Center
  - Children's National Medical Center


In [49]:
#download census boundary data

!pip install pygris

print("✓ Library installed!")

✓ Library installed!
